In [1]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  #'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith',
                  ]

MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [4]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt

_________

### Functions

In [5]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores


-------------

## Prep data

In [6]:
# Read Data
df = pd.read_pickle(DF_PATH)

### Balance classes

In [7]:
# Check the total samples of roles
roles_df = df["DevType"].copy()
roles_df.sum(axis=0)

Academic researcher                               1708
Data or business analyst                          1658
Data scientist or machine learning specialist     2460
Database administrator                            1210
DevOps specialist                                 3056
Developer, QA or test                             1135
Developer, back-end                              17084
Developer, desktop or enterprise applications     4845
Developer, embedded applications or devices       2138
Developer, front-end                              8932
Developer, full-stack                            20655
Developer, game or graphics                        899
Developer, mobile                                 4751
Engineer, data                                    1941
Scientist                                         1046
System administrator                              2069
dtype: int64

In [8]:
# Resample roles
samples_per_class = 1200
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()

    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0)

    resampled_roles.append(sub_df)

In [9]:
# Construct dfs
roles_df = pd.concat(resampled_roles)
df = df.loc[roles_df.index].copy()

In [10]:
roles_df.sum(axis=0)

Academic researcher                              2280
Data or business analyst                         1965
Data scientist or machine learning specialist    2576
Database administrator                           1765
DevOps specialist                                2170
Developer, QA or test                            1514
Developer, back-end                              5710
Developer, desktop or enterprise applications    2690
Developer, embedded applications or devices      1773
Developer, front-end                             2614
Developer, full-stack                            5602
Developer, game or graphics                      1441
Developer, mobile                                2155
Engineer, data                                   2046
Scientist                                        1910
System administrator                             2110
dtype: int64

## Split to train and test

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('DevType', axis=1),
                                                    df['DevType'],
                                                    random_state=0)

C:\Users\Abdelrahman Ragab\AppData\Local\Temp\ipykernel_27520\2536690730.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test, Y_train, Y_test = train_test_split(df.drop('DevType', axis=1),


_________

# Train models

### Initialize MLflow

In [12]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

### 1. Vanilla Forest 

In [13]:
rf_clf = make_pipeline(RobustScaler(),
                       PCA(n_components=0.95),
                       RandomForestClassifier(n_jobs=8,
                                              verbose=1,
                                              random_state=0))

rf_clf.fit(X_train.values, Y_train.values)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    4.5s finished


Pipeline(steps=[('robustscaler', RobustScaler()),
                ('pca', PCA(n_components=0.95)),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=8, random_state=0, verbose=1))])

In [14]:
# Evaluate on train set
predictions =  pd.DataFrame(rf_clf.predict(X_train.values),
                            columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


In [15]:
# Evaluate on test set
predictions =  pd.DataFrame(rf_clf.predict(X_test.values),
                            columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


In [16]:
print(test_scores.mean())
test_scores.sort_values("precision_score")

accuracy_score     92.647500
precision_score    93.886250
recall_score       50.186875
f1_score           64.286875
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
"Developer, full-stack",83.42,76.97,61.44,68.34
"Developer, back-end",79.35,83.76,38.44,52.70
Data scientist or machine learning specialist,93.81,87.90,62.81,73.27
"Developer, front-end",90.00,90.73,34.01,49.47
Academic researcher,94.94,93.59,60.43,73.44
"Developer, desktop or enterprise applications",90.94,94.07,34.53,50.51
Scientist,97.15,95.66,75.76,84.55
Data or business analyst,94.52,95.93,48.26,64.22
"Developer, mobile",93.85,96.14,46.63,62.80
DevOps specialist,92.83,96.57,36.89,53.39


### Log

In [17]:
## Log
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices": X_test.index.tolist(),
                "features_names": X_train.columns.droplevel(0).tolist(),
                "targets_names": Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [18]:
# Model
model = {"model_description": "Random Forest: with PCA - Basic",
         "model_details": str(rf_clf),
         "model_object": rf_clf}

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [19]:
# Performance details
classes_metrics = {"train_scores": train_scores,
                   "test_scores": test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [20]:
# Start a new run and track
with mlflow.start_run(experiment_id=exp.experiment_id,
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)

    # Track metrics
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score)

______________

### 2. Hyper parameter tuning

In [21]:
hpt_rf_clf = make_pipeline(RobustScaler(),
                           PCA(),
                           RandomForestClassifier(n_jobs=8,
                                                  verbose=1,
                                                  random_state=0))

In [22]:
# list(hpt_rf_clf.get_params().keys())
tuned_parameters = [{
    'pca__n_components': [0.7, 0.85, 0.95],
    'randomforestclassifier__n_estimators': [250, 500],
    'randomforestclassifier__max_depth':    [3, 10, None],
}]


In [23]:
hpt_rf_clf = GridSearchCV(hpt_rf_clf, tuned_parameters)
hpt_rf_clf.fit(X_train.values, Y_train.values)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    4.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 235 out of 250 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 250 out

[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 352 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    4.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 235 out of 250 | elapsed:    4.8s remaining:    0.2s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    5.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elap

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    3.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    3.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed: 

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    5.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   12.5s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   14.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.8s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    5.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   12.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   14.1s finished
[Parallel(n

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 352 tasks      | elapsed:    7.8s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   11.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 352 tasks      | elapsed:    7.9s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   11.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.7s finished


GridSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('pca', PCA()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(n_jobs=8,
                                                               random_state=0,
                                                               verbose=1))]),
             param_grid=[{'pca__n_components': [0.7, 0.85, 0.95],
                          'randomforestclassifier__max_depth': [3, 10, None],
                          'randomforestclassifier__n_estimators': [250, 500]}])

In [24]:
hpt_rf_clf.best_params_

{'pca__n_components': 0.7,
 'randomforestclassifier__max_depth': 3,
 'randomforestclassifier__n_estimators': 250}

In [25]:
# Evaluate on test set
predictions =  pd.DataFrame(hpt_rf_clf.predict(X_train.values),
                            columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.0s finished
C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_classification.py:13

In [26]:
# Evaluate on test set
predictions =  pd.DataFrame(hpt_rf_clf.predict(X_test.values),
                            columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.5s finished
C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_classification.py:13

In [27]:
print(test_scores.mean())
test_scores

accuracy_score     86.869375
precision_score     0.000000
recall_score        0.000000
f1_score            0.000000
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,88.42,0.0,0.0,0.0
Data or business analyst,89.81,0.0,0.0,0.0
Data scientist or machine learning specialist,86.50,0.0,0.0,0.0
Database administrator,90.79,0.0,0.0,0.0
DevOps specialist,88.88,0.0,0.0,0.0
"Developer, QA or test",91.77,0.0,0.0,0.0
"Developer, back-end",70.08,0.0,0.0,0.0
"Developer, desktop or enterprise applications",86.60,0.0,0.0,0.0
"Developer, embedded applications or devices",91.33,0.0,0.0,0.0
"Developer, front-end",85.60,0.0,0.0,0.0


## Log

In [28]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [29]:
# Model
model = {"model_description": "Random Forest: with PCA + Hyperparamter tuning",
         "model_details": str(hpt_rf_clf),
         "model_object": hpt_rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [30]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [31]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
       print( mlflow.log_metric(metric, score)) 
    

None
None
None
None


In [32]:
runs = mlflow.search_runs([exp.experiment_id])
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.recall_score,metrics.f1_score,metrics.precision_score,metrics.accuracy_score,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.name
0,79f7737b4ea44cc2b57b18ad1e6b256a,0,FINISHED,file:///C:/Users/Abdelrahman%20Ragab/Downloads...,2022-08-19 20:59:16.951000+00:00,2022-08-19 20:59:16.970000+00:00,50.186875,64.286875,93.886250,92.647500,LOCAL,Abdelrahman Ragab,Random Forest: with PCA + Hyperparamter tuning,C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\l...
1,cc159051f86b415ba8c387c102682310,0,FINISHED,file:///C:/Users/Abdelrahman%20Ragab/Downloads...,2022-08-19 20:49:49.918000+00:00,2022-08-19 20:49:52.112000+00:00,50.186875,64.286875,93.886250,92.647500,LOCAL,Abdelrahman Ragab,Random Forest: with PCA - Basic,C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\l...
2,d7ed955116154fa9b5772997229ee015,0,FINISHED,file:///C:/Users/Abdelrahman%20Ragab/Downloads...,2022-08-19 20:48:54.052000+00:00,2022-08-19 20:48:54.071000+00:00,29.890000,38.613125,62.420625,88.939375,LOCAL,Abdelrahman Ragab,Baseline model: Logistic Regression,C:\Users\Abdelrahman Ragab\anaconda3\envs\ML\l...
